In [1]:
# TOKEN = 44cd49327beab7f675215e6cc36a0547d4828f943c3f875a43fd2443
import tushare as tu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def evaluate_investment(source_data, tittle, time='交易日期'):
    temp = source_data.copy()
    # ===新建一个dataframe保存回测指标
    results = pd.DataFrame()

    # ===计算累积净值
    results.loc[0, '累积净值'] = round(temp[tittle].iloc[-1], 2)

    # ===计算年化收益
    annual_return = (temp[tittle].iloc[-1]) ** (
            '1 days 00:00:00' / (temp[time].iloc[-1] - temp[time].iloc[0]) * 365) - 1
    results.loc[0, '年化收益'] = str(round(annual_return * 100, 2)) + '%'

    # ===计算最大回撤，最大回撤的含义：《如何通过3行代码计算最大回撤》https://mp.weixin.qq.com/s/Dwt4lkKR_PEnWRprLlvPVw
    # 计算当日之前的资金曲线的最高点
    temp['max2here'] = temp[tittle].expanding().max()
    # 计算到历史最高值到当日的跌幅，drowdwon
    temp['dd2here'] = temp[tittle] / temp['max2here'] - 1
    # 计算最大回撤，以及最大回撤结束时间
    end_date, max_draw_down = tuple(temp.sort_values(by=['dd2here']).iloc[0][[time, 'dd2here']])
    # 计算最大回撤开始时间
    start_date = temp[temp[time] <= end_date].sort_values(by=tittle, ascending=False).iloc[0][
        time]
    # 将无关的变量删除
    temp.drop(['max2here', 'dd2here'], axis=1, inplace=True)
    results.loc[0, '最大回撤'] = format(max_draw_down, '.2%')
    results.loc[0, '最大回撤开始时间'] = str(start_date)
    results.loc[0, '最大回撤结束时间'] = str(end_date)

    # ===年化收益/回撤比：我个人比较关注的一个指标
    results.loc[0, '年化收益/回撤比'] = round(annual_return / abs(max_draw_down), 2)

    return results.T


In [3]:
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
pd.set_option('display.max_rows', 5000)  # 最多显示数据的行数

In [4]:
# 读取数据
df_big = pd.read_csv('大小盘轮动/sh000300.csv', encoding='gbk', parse_dates=['candle_end_time'])
df_small = pd.read_csv('大小盘轮动/sz399006.csv', encoding='gbk', parse_dates=['candle_end_time'])

In [5]:
# 设置参数
trade_rate = 0.6 / 10000  # 场内基金万分之0.6，买卖手续费相同，无印花税
momentum_days = 20  # 计算多少天的动量

In [6]:
# 计算大小盘每天的涨跌幅amplitude
df_big['big_amp'] = df_big['close'] / df_big['close'].shift(1) - 1
df_small['small_amp'] = df_small['close'] / df_small['close'].shift(1) - 1

In [7]:
# 重命名行
df_big.rename(columns={'open': 'big_open', 'close': 'big_close'}, inplace=True)
df_small.rename(columns={'open': 'small_open', 'close': 'small_close'}, inplace=True)

In [8]:
# 合并数据
df = pd.merge(left=df_big[['candle_end_time', 'big_open', 'big_close', 'big_amp']], left_on=['candle_end_time'],
              right=df_small[['candle_end_time', 'small_open', 'small_close', 'small_amp']],
              right_on=['candle_end_time'], how='left')

In [9]:
# 计算N日的动量momentum
df['big_mom'] = df['big_close'].pct_change(periods=momentum_days)
df['small_mom'] = df['small_close'].pct_change(periods=momentum_days)

In [10]:
# 风格变换条件
df.loc[df['big_mom'] > df['small_mom'], 'style'] = 'big'
df.loc[df['big_mom'] < df['small_mom'], 'style'] = 'small'

In [11]:
# 相等时维持原来的仓位。
df['style'].fillna(method='ffill', inplace=True)

In [12]:
# 收盘才能确定风格，实际的持仓pos要晚一天。
df['pos'] = df['style'].shift(1)
df

,candle_end_time,big_open,big_close,big_amp,small_open,small_close,small_amp,big_mom,small_mom,style,pos
0,2005-04-08,984.660,1003.450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005-04-11,1003.880,995.420,-8.002392e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-04-12,993.710,978.700,-1.679693e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005-04-13,987.950,1000.900,2.268315e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005-04-14,1004.640,986.970,-1.391747e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2005-04-15,982.610,974.080,-1.306017e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2005-04-18,970.910,963.770,-1.058435e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2005-04-19,962.920,965.890,2.199695e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2005-04-20,964.150,950.870,-1.555042e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2005-04-21,948.860,943.980,-7.245996e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# 删除持仓为nan的天数（创业板2010年才有）
df.dropna(subset=['pos'], inplace=True)

In [14]:
# 计算策略的整体涨跌幅strategy_amp
df.loc[df['pos'] == 'big', 'strategy_amp'] = df['big_amp']
df.loc[df['pos'] == 'small', 'strategy_amp'] = df['small_amp']
df

,candle_end_time,big_open,big_close,big_amp,small_open,small_close,small_amp,big_mom,small_mom,style,pos,strategy_amp
1275,2010-07-06,2504.869,2562.902,0.019999,859.258,893.718,0.036474,-0.063293,-0.104844,big,big,0.019999
1276,2010-07-07,2561.106,2580.477,0.006857,896.532,908.549,0.016595,-0.059727,-0.115923,big,big,0.006857
1277,2010-07-08,2591.509,2575.921,-0.001766,911.997,905.509,-0.003346,-0.044439,-0.153309,big,big,-0.001766
1278,2010-07-09,2578.555,2647.104,0.027634,903.496,932.147,0.029418,-0.019350,-0.126511,big,big,0.027634
1279,2010-07-12,2647.420,2676.220,0.010999,931.530,930.170,-0.002121,-0.038069,-0.136992,big,big,0.010999
1280,2010-07-13,2649.658,2634.593,-0.015554,924.948,931.080,0.000978,-0.041974,-0.166802,big,big,-0.015554
1281,2010-07-14,2640.925,2653.609,0.007218,929.749,922.362,-0.009363,-0.038152,-0.178076,big,big,0.007218
1282,2010-07-15,2650.334,2608.519,-0.016992,920.046,892.557,-0.032314,-0.048934,-0.161792,big,big,-0.016992
1283,2010-07-16,2597.000,2616.128,0.002917,885.067,892.298,-0.000290,-0.029686,-0.107949,big,big,0.002917
1284,2010-07-19,2592.087,2682.472,0.025360,881.656,908.049,0.017652,-0.035312,-0.127549,big,big,0.025360


In [15]:
# 调仓时间
df.loc[df['pos'] != df['pos'].shift(1), 'trade_time'] = df['candle_end_time']

In [16]:
# 将调仓日的涨跌幅修正为开盘价买入涨跌幅（并算上交易费用，没有取整数100手，所以略有误差）
df.loc[(df['trade_time'].notnull()) & (df['pos'] == 'big'), 'strategy_amp_adjust'] = df['big_close'] / (
        df['big_open'] * (1 + trade_rate)) - 1
df.loc[(df['trade_time'].notnull()) & (df['pos'] == 'small'), 'strategy_amp_adjust'] = df['small_close'] / (
        df['small_open'] * (1 + trade_rate)) - 1
df.loc[df['trade_time'].isnull(), 'strategy_amp_adjust'] = df['strategy_amp']

In [17]:
# 扣除卖出手续费
df.loc[(df['trade_time'].shift(-1).notnull()), 'strategy_amp_adjust'] = (1 + df[
    'strategy_amp']) * (1 - trade_rate) - 1
del df['strategy_amp'], df['style']

In [18]:
df.reset_index(drop=True, inplace=True)
# 计算净值
df['big_net'] = df['big_close'] / df['big_close'][0]
df['small_net'] = df['small_close'] / df['small_close'][0]
df['strategy_net'] = (1 + df['strategy_amp_adjust']).cumprod()

In [19]:
# 评估策略的好坏
res = evaluate_investment(df, 'strategy_net', time='candle_end_time')
print(res)

                            0
累积净值                    10.18
年化收益                   24.54%
最大回撤                  -48.13%
最大回撤开始时间  2015-06-03 00:00:00
最大回撤结束时间  2015-08-26 00:00:00
年化收益/回撤比                 0.51


In [20]:
big_close_rate = df['big_close'].pct_change(periods = 1)
df['big_amp'] == big_close_rate

0       False
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27       True
28       True
29       True
30       True
31       True
32       True
33       True
34       True
35       True
36       True
37       True
38       True
39       True
40       True
41       True
42       True
43       True
44       True
45       True
46       True
47       True
48       True
49       True
50       True
51       True
52       True
53       True
54       True
55       True
56       True
57       True
58       True
59       True
60       True
61       True
62       True
63       True
64       True
65       True
66       True
67       True
68       True
69       True
70       True
71    